In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyproj import Proj, Transformer

driver = webdriver.Chrome()
driver.maximize_window()
dong = "예시동"
df = pd.DataFrame(columns=['동', 'utm_x', 'utm_y', '타입', '가격', '면적', '층'])

zone_number = 52
northern = True

# 좌표 변환기 설정
utm_proj = Proj(proj='utm', zone=zone_number, ellps='WGS84', south=not northern)
wgs84_proj = Proj(proj='latlong', datum='WGS84')
transformer_to_utm = Transformer.from_proj(wgs84_proj, utm_proj, always_xy=True)
transformer = Transformer.from_proj(utm_proj, wgs84_proj, always_xy=True)
meter_per_px = 52 / 55
latlon_df = pd.read_csv('./files/seogu_filt.csv', header=None, names=['lat', 'lon'])

for idx, row in latlon_df.iterrows():
    lat, lon = row['lat'], row['lon']
    origin_x, origin_y = transformer_to_utm.transform(lon, lat)

    print(f"[{idx+1}] 이동: {lat}, {lon}")
    driver.get(f"https://new.land.naver.com/offices?ms={lat},{lon},18&a=SG:SMS:GJCG:APTHGJ:GM:TJ&b=B2&e=RETAIL")
    time.sleep(2)

    try:
        pin_elements = driver.find_elements(By.CSS_SELECTOR, "a.map_cluster--mix:not(.is-outside)")
    except:
        print("핀 로딩 실패")
        continue


    for i, pin in enumerate(pin_elements):
        try:
            map_element = driver.find_element(By.CSS_SELECTOR, "div#map.map_panel")
            list_width = driver.find_element(By.CSS_SELECTOR, "div#listContents1").size['width']
            my_x = map_element.size['width'] / 2 + list_width / 2
            my_y = map_element.size['height'] / 2

            target_style = pin.get_attribute("style")
            target_x = float(target_style.split("left:")[1].split("px")[0].strip())
            target_y = float(target_style.split("top:")[1].split("px")[0].strip())

            # 좌표가 비정상적으로 튀는 경우 새로고침
            if abs(target_x) > 10000 or abs(target_y) > 10000:
                print(f"[좌표 이상] target_x: {target_x}, target_y: {target_y} → 새로고침")
                driver.refresh()
                time.sleep(3)
                continue

            # 기존 조건도 유지
            if target_x < 380 or target_y < -30:
                print(f"[건너뜀] target_x: {target_x}, target_y: {target_y}")
                continue


            distance_x = (target_x - my_x) / 2 * meter_per_px
            distance_y = (target_y - my_y) / 2 * meter_per_px

            utm_x = origin_x + distance_x
            utm_y = origin_y - distance_y

            print(f"  ↳ 핀 {i+1} → UTM: {utm_x}, {utm_y}")

            c_btns = driver.find_elements(By.CSS_SELECTOR, "button.btn_close")

            for btn in c_btns:
                label = btn.get_attribute("aria-label")
                if label == "상세페이지 닫기":
                    btn.click()
                    break  # 클릭한 후 반복 종료

            pin.click()



            time.sleep(1)

            try:
                body = driver.find_element(By.CSS_SELECTOR, 'div.item_list.item_list--article')
                while True:
                    body.send_keys(Keys.END)
                    time.sleep(1)
                    driver.find_element(By.CSS_SELECTOR, 'div.loader')  # 로딩 대기
            except:
                pass


            results = []  # 결과 저장용 리스트

            items = driver.find_elements(By.CSS_SELECTOR, "div.item")

            for item in items:
                item.click()
                time.sleep(0.5)

                ty = driver.find_elements(By.CSS_SELECTOR, "h4.info_title")
                title = ty[1].text

                pay = driver.find_element(By.CSS_SELECTOR, "div.info_article_price ")
                price = pay.text

                rows = driver.find_elements(By.CSS_SELECTOR, "tr.info_table_item")

                data = {}
                stop_flag = False
                for row in rows:
                    if stop_flag:
                        break
                    ths = row.find_elements(By.CSS_SELECTOR, "th")
                    tds = row.find_elements(By.CSS_SELECTOR, "td")

                    for i in range(len(ths)):
                        key = ths[i].text.strip()
                        value = tds[i].text.strip() if i < len(tds) else ""

                        data[key] = value

                        if key == "건물번호":
                            stop_flag = True
                            break

                # 제목과 가격 추가
                data["제목"] = title
                data["가격"] = price

                # 데이터 누적
                data["동"] = dong
                data["utm_x"] = utm_x
                data["utm_y"] = utm_y

                df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

        except Exception as e:
            print(f"[클릭 실패] 좌표: x={target_x}, y={target_y} / 오류: {e}")
            continue

# 반복문 끝난 후 위경도 변환 및 저장 --------------------

# -------------------- UTM → 위경도 변환 --------------------
utm_coords = list(zip(df['utm_x'], df['utm_y']))
latlon_list = [transformer.transform(x, y) for x, y in utm_coords]

df['경도'] = [round(lon, 7) for lon, lat in latlon_list]
df['위도'] = [round(lat, 7) for lon, lat in latlon_list]

# -------------------- CSV 저장 --------------------
df.drop(columns=["매물설명", "중개사", "중개보수", "융자금", "난방(방식/연료)", "총주차대수", "주차가능여부", "건축물 용도", "면적", "층", "타입"], inplace=True)
df.to_csv(f"./data/03_부동산/{dong}_부동산.csv", encoding='utf-8-sig', index=False)
df[['위도', '경도']].to_csv(f"./data/04_부동산_test/{dong}_test.csv", encoding='utf-8-sig', index=False, header=False)

print("CSV 저장 완료")

[1] 이동: 35.1149426, 126.8284109
  ↳ 핀 1 → UTM: 302270.0898990034, 3888096.2027922655


C:\Users\smhrd\AppData\Local\Temp\ipykernel_6076\1367242853.py:138: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)


[2] 이동: 35.1149426, 126.8324109
  ↳ 핀 1 → UTM: 302634.6744894122, 3888088.2577608363
[3] 이동: 35.1149426, 126.8364109
  ↳ 핀 1 → UTM: 302646.76966076775, 3888080.327398637
[클릭 실패] 좌표: x=773.64, y=54.4778 / 오류: Message: element click intercepted: Element <a href="javascript:void(0);" role="button" class="map_cluster--mix" aria-pressed="false" aria-hidden="false" id="03200132203320LGEOHASH_MIX_ARTICLE" data-nclk="MAP.rectangleAmark" style="width: 41.27px; height: 41.27px; top: 54.4778px; left: 773.64px;">...</a> is not clickable at point (48, 238). Other element would receive the click: <iframe id="land_panel_da_tgtLREC" frameborder="no" scrolling="no" tabindex="0" name="" title="AD" style="width: 100%; height: 80px; visibility: inherit; border: 0px; vertical-align: bottom;"></iframe>
  (Session info: chrome=135.0.7049.85)
Stacktrace:
	GetHandleVerifier [0x00007FF6F5B55335+78597]
	GetHandleVerifier [0x00007FF6F5B55390+78688]
	(No symbol) [0x00007FF6F59091AA]
	(No symbol) [0x00007FF6F596708

In [ ]:
df.drop(columns=["매물설명", "중개사", "중개보수", "융자금", "난방(방식/연료)", "총주차대수", "주차가능여부", "건축물 용도", "면적", "층", "타입"], inplace=True)
df

,동,utm_x,utm_y,가격,소재지,매물특징,계약/전용면적,해당층/총층,입주가능일,권리금,...,현재업종,추천업종,용도지역,주구조,사용승인일,매물번호,제목,총사무실수,경도,위도
0,예시동,305239.925466,3.890576e+06,"월세2,500/160",광주시 서구 금호동,금호 사거리 코너상가 1층 베이커리카페추천 내부깨끗,178.7㎡/135.55㎡(전용률76%),1/4층,즉시입주,없음,...,제2종근린생활시설,-,-,-,2018.02.05,2517440152,일반상가 1동1층,NaN,126.862223,35.139239
1,예시동,305239.925466,3.890576e+06,"월세2,500/160",광주시 서구 금호동,l 무권리 l 3면코너 l 브런치 베이커리 판매점 적합 l 현 공실,117㎡/117㎡(전용률100%),1/4층,즉시입주,NaN,...,-,-,-,-,2018.02.05,2516838316,일반상가1층,NaN,126.862223,35.139239
2,예시동,305239.925466,3.890576e+06,"월세2,500/160",광주시 서구 금호동,"1층 대로변 코너자리, 광고효과최상, 카페추천, 유동인구아주많음",105㎡/105㎡(전용률100%),1/4층,즉시입주,NaN,...,-,-,-,-,2018.02.05,2518753457,일반상가1층,NaN,126.862223,35.139239
3,예시동,305239.925466,3.890576e+06,"월세2,500/160",광주시 서구 금호동,"무권리, 건물깨끗, 3면 통유리, 브런치 베이커리 카페 판매점 추천",122.31㎡/122.31㎡(전용률100%),1/4층,즉시입주,NaN,...,-,-,-,-,2018.02.05,2515654500,일반상가1층,NaN,126.862223,35.139239
4,예시동,305229.564038,3.890569e+06,"월세3,000/110",광주시 서구 금호동,"내부 사무실 시설완비, 탕비실별도, 주차편리, 대로변 광고효과좋음",172.47㎡/172.47㎡(전용률100%),2/6층,즉시입주,NaN,...,NaN,NaN,NaN,NaN,2017.04.27,2519168018,중소형사무실2층,-,126.862111,35.139177
5,예시동,305229.564038,3.890569e+06,"월세3,000/110",광주시 서구 금호동,"마륵공원 근접 내부깨끗한 신축급 탕비실,창고 구비 주차편리 모든업종가",172㎡/172㎡(전용률100%),2/6층,즉시입주,NaN,...,NaN,NaN,NaN,NaN,2017.04.27,2516299252,대형사무실2층,-,126.862111,35.139177
6,예시동,305245.578085,3.890628e+06,"월세2,000/190",광주시 서구 금호동,"차 광택 현성업중 기술전수가 사정상 급임대 시,권저렴 조정가 부가세별도",132㎡/132㎡(전용률100%),1/3층,즉시입주 협의가능,"2,500만원",...,-,-,-,-,2005.02.01,2514210060,일반상가1층,NaN,126.862272,35.139708
7,예시동,305201.904038,3.890604e+06,월세500/50,광주시 서구 금호동,"무권리, 접근성좋음, 주차편리, 모든업종가.",47.22㎡/47.22㎡(전용률100%),1/3층,즉시입주 협의가능,NaN,...,-,-,-,-,1995.11.04,2518612041,일반상가1층,NaN,126.861799,35.139484
8,예시동,305170.846181,3.890611e+06,"월세4,000/140",광주시 서구 금호동,"l 운천로대로변 l 현공실 유동인구,광고효과최상 제1종근생",66㎡/60㎡(전용률91%),1/1층,즉시입주 협의가능,NaN,...,-,-,-,-,1994.02.04,2516754305,일반상가1층,NaN,126.861457,35.139536
9,예시동,305170.846181,3.890611e+06,"월세4,000/140",광주시 서구 금호동,"무권리, 대단지아파트, 유동인구많음, 광고효과좋음, 소매점 판매점 추천",60.3㎡/60.3㎡(전용률100%),1/1층,즉시입주,NaN,...,-,-,-,-,1994.02.04,2515668151,일반상가1층,NaN,126.861457,35.139536


In [ ]:
len(df)

In [ ]:
len(pin_elements)